## Setting

In [1]:
import pandas as pd
import sys
import os
import numpy as np
import json
from datetime import datetime
from clust.pipeline import data_pipeline 

sys.path.append("../")
sys.path.append("../..")
sys.path.append("../../..")
sys.path.append("../../../..")
sys.path.append("../keti-datamanager")
sys.path.append("../keti-datamanager/backend")

In [ ]:
from clust.ingestion import influx_client

clust_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"lRO_RjCuIpdgzpwx9bwhU_0MhyFflFA_fcPRVy6nGLEmJPnFAjB4zt-xbUh6UCaAA_36jCpZ3s9KCj9KkggcRA==",
        "org":"clust"
}

cedp_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"U9QaDrozQDsGdnxpZkkq5NJ1fwzDefEDtR7a_vAaylcxR6LsY1wCc36QVVqDah6ytctNYFM__CfzFuzQ8K4Niw==",
        "org":"cedp"
}

agridata_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"jUOMSF1jfAg7Kh-DscoswyBfMe6VAN1XdEabpYSTDbkeMw8SdNf1KFmQD0bomJC2rmLy0dwvwLrnySZxCfoMUQ==",
        "org":"cedp"
}

db_client = influx_client.InfluxClient(cedp_config)

In [15]:
# backend 모듈 import (현재 경로의 code 폴더에서 가져오기)
import sys
sys.path.insert(0, './code')

# 기존 모듈이 있으면 reload
import importlib
if 'backend_data_integration' in sys.modules:
    import backend_data_integration
    importlib.reload(backend_data_integration)

from backend_data_integration import (
    get_secondary_data_list, 
    get_all_secondary_data_list,
    get_fields,
    process_data_integration
)
from clust.data import data_interface

print("모듈 import 완료 (경로: ./code/backend_data_integration.py)")

모듈 import 완료 (경로: ./code/backend_data_integration.py)


## 1. 기준 데이터 조회 및 선택
- 기준 데이터: farm_protectedHoriticulture_westernVegetables_paprika / environment_knu_n
- 태그: FRM_ID: F00012, ZN_ID: 1구역

In [7]:
# 기준 데이터 정보 설정
base_bucket = "farm_protectedHorticulture_westernVegetables_paprika"
base_measurement = "environment_knu_n"
base_tag_key1 = "FRM_ID"
base_tag_value1 = "F00012"
base_tag_key2 = "ZN_ID"
base_tag_value2 = "1구역"

# 기준 데이터 조회 (샘플로 1일치만)
base_data_query = {
    'bucket_name': base_bucket,
    'ms_name': base_measurement,
    'start_time': '2021-07-01 00:00:00',
    'end_time': '2021-07-02 00:00:00',
    'tags': {base_tag_key1: base_tag_value1, base_tag_key2: base_tag_value2}
}

print(f"조회 설정:")
print(f"  Bucket: {base_bucket}")
print(f"  Measurement: {base_measurement}")
print(f"  Tags: {base_tag_key1}={base_tag_value1}, {base_tag_key2}={base_tag_value2}")

base_data_sample = data_interface.get_data_result("ms_by_time", db_client, base_data_query)
print(f"\n기준 데이터 샘플 크기: {len(base_data_sample) if base_data_sample is not None else 0}")
print(f"기준 데이터 컬럼: {list(base_data_sample.columns) if base_data_sample is not None else []}")
base_data_sample.head() if base_data_sample is not None else None

조회 설정:
  Bucket: farm_protectedHorticulture_westernVegetables_paprika
  Measurement: environment_knu_n
  Tags: FRM_ID=F00012, ZN_ID=1구역

기준 데이터 샘플 크기: 144
기준 데이터 컬럼: ['EXT_TMP', 'EXT_WIND_DIRECT', 'EXT_WIND_SPEED', 'INT_CO2', 'INT_REL_HD', 'INT_TMP', 'MAX_EXT_TMP', 'MIN_EXT_TMP']


,EXT_TMP,EXT_WIND_DIRECT,EXT_WIND_SPEED,INT_CO2,INT_REL_HD,INT_TMP,MAX_EXT_TMP,MIN_EXT_TMP
time,,,,,,,,
2021-07-01 00:00:00+00:00,19.50,90.0,0.00,558.75,80.33,22.25,20.2,19.7
2021-07-01 00:10:00+00:00,19.87,90.0,0.30,394.20,63.42,13.31,20.3,19.5
2021-07-01 00:20:00+00:00,19.75,90.0,0.31,395.00,63.82,13.22,20.0,19.5
2021-07-01 00:30:00+00:00,19.69,90.0,0.15,399.50,63.63,13.21,19.8,19.5
2021-07-01 00:40:00+00:00,19.72,90.0,0.04,399.80,63.53,13.23,19.9,19.6


## 2. 기준 데이터와 유사한 데이터 리스트 조회
- 기준 데이터의 시간 범위를 포함하는 데이터 찾기
- 스코어와 위치 유사도 비교

In [8]:
# 유사 데이터 리스트 조회
primary_data = {
    'bucket': base_bucket,
    'measurements': base_measurement
}

# 모든 유사 데이터 조회 (시간 필터링 없음)
print("유사 데이터 조회 중...")
all_secondary_list = get_all_secondary_data_list(primary_data, base_tag_key1, base_tag_value1)
print(f"전체 유사 데이터 개수: {len(all_secondary_list)}")

# 유사 데이터 리스트 출력
if all_secondary_list:
    print("\n=== 유사 데이터 리스트 (상위 10개) ===")
    for idx, data in enumerate(all_secondary_list[:10]):
        print(f"\n{idx+1}. Bucket: {data[0]}")
        print(f"   Measurement: {data[1]}")
        if len(data) > 2:
            print(f"   Tags: {data[2]}")
        if len(data) > 3:
            print(f"   Score: {data[3]}")
        if len(data) > 4:
            print(f"   Spatial Similarity: {data[4]}")
        if len(data) > 5:
            print(f"   Keyword Similarity: {data[5]}")
else:
    print("유사 데이터가 없습니다.")

유사 데이터 조회 중...
전체 유사 데이터 개수: 6

=== 유사 데이터 리스트 (상위 10개) ===

1. Bucket: farm_protectedHorticulture_vegetables_cucumber
   Measurement: environment_knu_sgh
   Tags: {'FARM_ID_CODE': 'A00259'}
   Score: 0.75
   Spatial Similarity: False
   Keyword Similarity: False

2. Bucket: farm_protectedHorticulture_vegetables_cucumber
   Measurement: yield_knu_n
   Tags: {'FARM_ID_CODE': 'A00259'}
   Score: 0.80
   Spatial Similarity: False
   Keyword Similarity: True

3. Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
   Measurement: environment_knu_n
   Tags: {'FRM_ID': 'F00013', 'ZN_ID': '1구역'}
   Score: 0.82
   Spatial Similarity: False
   Keyword Similarity: False

4. Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
   Measurement: environment_knu_sgh
   Tags: {'FARM_ID_CODE': 'A00259'}
   Score: 0.87
   Spatial Similarity: False
   Keyword Similarity: True

5. Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
   Measurement: yield_knu_n
   Tags: {'FARM_ID

## 3. 통합할 데이터 선택
- 목표 데이터: farm_protectedHoriticulture_fruitsAndVegetables_tomato / environment_knu_sgh
- 리스트에 있으면 선택, 없으면 유사도가 높은 데이터 선택

In [26]:
# 목표 데이터 정보 (변경됨)
target_bucket = "farm_protectedHorticulture_fruitsAndVegetables_tomato"
target_measurement = "environment_knu_n"  # environment_knu_sgh에서 변경
target_tag_key1 = "FRM_ID"
target_tag_value1 = "F00013"
target_tag_key2 = "ZN_ID"
target_tag_value2 = "1구역"

print(f"목표 데이터 (변경됨):")
print(f"  Bucket: {target_bucket}")
print(f"  Measurement: {target_measurement}")
print(f"  Tags: {target_tag_key1}={target_tag_value1}, {target_tag_key2}={target_tag_value2}")

# 리스트에서 목표 데이터 찾기
selected_data = None
for data in all_secondary_list:
    if data[0] == target_bucket and data[1] == target_measurement:
        # 태그도 확인
        if len(data) > 2 and isinstance(data[2], dict):
            tags = data[2]
            if tags.get(target_tag_key1) == target_tag_value1 and tags.get(target_tag_key2) == target_tag_value2:
                selected_data = data
                print(f"\n✓ 목표 데이터를 리스트에서 찾았습니다!")
                print(f"  Bucket: {data[0]}")
                print(f"  Measurement: {data[1]}")
                print(f"  Tags: {data[2]}")
                if len(data) > 3:
                    print(f"  Score: {data[3]}")
                break

# 리스트에 없으면 직접 생성
if selected_data is None:
    print("\n목표 데이터가 리스트에 없습니다. 직접 설정합니다.")
    selected_data = [
        target_bucket,
        target_measurement,
        {target_tag_key1: target_tag_value1, target_tag_key2: target_tag_value2}
    ]
    print(f"\n선택된 데이터:")
    print(f"  Bucket: {selected_data[0]}")
    print(f"  Measurement: {selected_data[1]}")
    print(f"  Tags: {selected_data[2]}")

if selected_data:
    selected_bucket = selected_data[0]
    selected_measurement = selected_data[1]
    selected_tags = selected_data[2] if len(selected_data) > 2 else {}
    print(f"\n최종 선택된 통합 데이터:")
    print(f"  Bucket: {selected_bucket}")
    print(f"  Measurement: {selected_measurement}")
    print(f"  Tags: {selected_tags}")
else:
    print("\n선택 가능한 데이터가 없습니다.")

목표 데이터 (변경됨):
  Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
  Measurement: environment_knu_n
  Tags: FRM_ID=F00013, ZN_ID=1구역

✓ 목표 데이터를 리스트에서 찾았습니다!
  Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
  Measurement: environment_knu_n
  Tags: {'FRM_ID': 'F00013', 'ZN_ID': '1구역'}
  Score: 0.82

최종 선택된 통합 데이터:
  Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
  Measurement: environment_knu_n
  Tags: {'FRM_ID': 'F00013', 'ZN_ID': '1구역'}


## 4. 통합할 컬럼 조회 및 선택
- 기준 데이터와 통합 데이터의 모든 컬럼 선택

In [19]:
# 기준 데이터의 필드 조회
print("기준 데이터 필드 조회 중...")
base_fields = get_fields(base_bucket, base_measurement, base_tag_key1, base_tag_value1)
print(f"기준 데이터 필드 ({len(base_fields)}개):")
print(base_fields)

# 선택된 데이터의 필드 조회
if selected_data:
    # 선택된 데이터의 태그 정보 추출
    selected_tag_key = None
    selected_tag_value = None
    if len(selected_data) > 2 and selected_tags:
        if isinstance(selected_tags, dict) and len(selected_tags) > 0:
            selected_tag_key = list(selected_tags.keys())[0]
            selected_tag_value = selected_tags[selected_tag_key]
    
    print(f"\n통합 데이터 필드 조회 중...")
    selected_fields = get_fields(selected_bucket, selected_measurement, selected_tag_key, selected_tag_value)
    print(f"통합 데이터 필드 ({len(selected_fields)}개):")
    print(selected_fields)
    
    # 모든 컬럼 사용하지 않고 각 데이터의 원본 필드만 사용
    # 중복되는 필드는 하나만 유지
    print(f"\n=== 통합 전략 ===")
    print(f"기준 데이터 필드: {base_fields}")
    print(f"통합 데이터 필드: {selected_fields}")
    print(f"\n각 데이터의 원본 필드를 그대로 사용합니다.")
else:
    print("선택된 데이터가 없어 필드를 조회할 수 없습니다.")

기준 데이터 필드 조회 중...
기준 데이터 필드 (8개):
['EXT_TMP', 'EXT_WIND_DIRECT', 'EXT_WIND_SPEED', 'INT_CO2', 'INT_REL_HD', 'INT_TMP', 'MAX_EXT_TMP', 'MIN_EXT_TMP']

통합 데이터 필드 조회 중...
통합 데이터 필드 (6개):
['ACC_RADIATION_INTENSITY', 'EXT_SLR_RADIATON', 'INT_CO2', 'INT_REL_HD', 'MID_INT_TMP', 'SLR_RAD']

=== 통합 전략 ===
기준 데이터 필드: ['EXT_TMP', 'EXT_WIND_DIRECT', 'EXT_WIND_SPEED', 'INT_CO2', 'INT_REL_HD', 'INT_TMP', 'MAX_EXT_TMP', 'MIN_EXT_TMP']
통합 데이터 필드: ['ACC_RADIATION_INTENSITY', 'EXT_SLR_RADIATON', 'INT_CO2', 'INT_REL_HD', 'MID_INT_TMP', 'SLR_RAD']

각 데이터의 원본 필드를 그대로 사용합니다.


## 5. 데이터 통합 실행
- 기준 데이터의 주기 사용
- 통합 시간 범위: 2021-07-01 ~ 2022-10-25

In [20]:
# 데이터 통합 요청 데이터 구성
# API 형식: [measurement, bucket, tag_key, tag_value, features]
# features를 빈 리스트로 전달하면 모든 필드를 사용
integration_request = {
    'base_data': [
        base_measurement,  # measurement
        base_bucket,       # bucket
        base_tag_key1,     # tag_key
        base_tag_value1,   # tag_value
        []                 # features - 빈 리스트로 모든 필드 사용
    ],
    'selected_data': [
        selected_measurement,  # measurement
        selected_bucket,       # bucket
        selected_tag_key if selected_tag_key else '',  # tag_key
        selected_tag_value if selected_tag_value else '',  # tag_value
        []                 # features - 빈 리스트로 모든 필드 사용
    ],
    'start_time': '2021-07-01',
    'end_time': '2022-10-25',
    'Period': 'data1'  # 기준 데이터의 주기 사용
}

print("=== 통합 요청 정보 ===")
print(f"기준 데이터: {base_bucket} / {base_measurement}")
print(f"  태그: {base_tag_key1}={base_tag_value1}")
print(f"  필드: 모든 필드 사용")
print(f"\n통합 데이터: {selected_bucket} / {selected_measurement}")
print(f"  태그: {selected_tag_key}={selected_tag_value if selected_tag_value else 'N/A'}")
print(f"  필드: 모든 필드 사용")
print(f"\n기간: {integration_request['start_time']} ~ {integration_request['end_time']}")
print(f"기준 주기: {integration_request['Period']}")

# 데이터 통합 실행
print("\n" + "="*50)
print("데이터 통합 실행 중...")
print("="*50)
integration_result = process_data_integration(integration_request)

print(f"\n통합 상태: {integration_result.get('status')}")
print(f"메시지: {integration_result.get('message')}")

if integration_result.get('status') == 'success':
    result = integration_result.get('result', {})
    print(f"\n=== 통합 결과 ===")
    print(f"원본 데이터셋 수: {result.get('original_datasets')}")
    print(f"전체 행 수: {result.get('total_original_rows'):,}")
    print(f"통합 주기: {result.get('integration_frequency')}")
    print(f"\n데이터셋별 행 수:")
    for ds_name, count in result.get('dataset_counts', {}).items():
        print(f"  - {ds_name}: {count:,}행")
    
    # 통합된 필드 정보
    if 'valid_features' in result:
        print(f"\n사용된 필드: {result.get('valid_features')}")
else:
    print(f"오류 발생: {integration_result.get('message')}")

=== 통합 요청 정보 ===
기준 데이터: farm_protectedHorticulture_westernVegetables_paprika / environment_knu_n
  태그: FRM_ID=F00012
  필드: 모든 필드 사용

통합 데이터: farm_protectedHorticulture_fruitsAndVegetables_tomato / environment_knu_sgh
  태그: FARM_ID_CODE=A00259
  필드: 모든 필드 사용

기간: 2021-07-01 ~ 2022-10-25
기준 주기: data1

데이터 통합 실행 중...

통합 상태: success
메시지: 데이터 통합이 완료되었습니다.

=== 통합 결과 ===
원본 데이터셋 수: 1
전체 행 수: 49,497
통합 주기: 240T

데이터셋별 행 수:
  - farm_protectedHorticulture_westernVegetables_paprika_environment_knu_n: 49,497행

사용된 필드: []


## 6. 통합 결과 저장 (CSV & JSON)
- 저장 경로: integration_results/
- 형식: CSV 파일 + JSON 메타데이터

In [21]:
import uuid
from datetime import datetime

# 저장 디렉토리 확인 및 생성
save_dir = "integration_results"
os.makedirs(save_dir, exist_ok=True)

# 저장 ID 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_id = f"integration_{timestamp}_{str(uuid.uuid4())[:8]}"

print(f"저장 ID: {save_id}")

# 통합 데이터 재조회 (실제 통합된 데이터)
if integration_result.get('status') == 'success':
    # 통합 데이터 다시 가져오기
    ms_list_info = [
        [base_bucket, base_measurement, {base_tag_key1: base_tag_value1, base_tag_key2: base_tag_value2}],
        [selected_bucket, selected_measurement, {selected_tag_key: selected_tag_value} if selected_tag_key else {}]
    ]
    
    ingestion_param = {
        'ms_list_info': ms_list_info,
        'start_time': '2021-07-01 00:00:00',
        'end_time': '2022-10-25 23:59:59'
    }
    
    print("\n통합 데이터 조회 중...")
    integrated_data = data_interface.get_data_result("multiple_ms_by_time", db_client, ingestion_param)
    
    if integrated_data is not None:
        print(f"조회된 데이터 타입: {type(integrated_data)}")
        
        # CSV 저장
        csv_path = os.path.join(save_dir, f"{save_id}.csv")
        
        if isinstance(integrated_data, dict):
            # 여러 데이터셋인 경우 모두 병합
            print(f"데이터셋 수: {len(integrated_data)}")
            all_dfs = []
            for key, df in integrated_data.items():
                print(f"  - {key}: {len(df)} 행")
                all_dfs.append(df)
            
            # 모든 데이터프레임을 하나로 병합
            if all_dfs:
                import pandas as pd
                df_to_save = pd.concat(all_dfs, axis=0, sort=False)
                print(f"\n병합된 데이터: {len(df_to_save)} 행")
            else:
                df_to_save = None
        else:
            df_to_save = integrated_data
        
        if df_to_save is not None and len(df_to_save) > 0:
            df_to_save.to_csv(csv_path, index=True)
            print(f"\n✓ CSV 저장 완료: {csv_path}")
            print(f"  저장된 행 수: {len(df_to_save):,}")
            print(f"  저장된 컬럼 수: {len(df_to_save.columns)}")
            
            # JSON 메타데이터 구성 (기존 형식과 동일하게)
            json_data = {
                "data1": {
                    "bucket": base_bucket,
                    "measurement": base_measurement,
                    "tagKey": base_tag_key1,
                    "tagValue": base_tag_value1
                },
                "data2": {
                    "bucket": selected_bucket,
                    "measurement": selected_measurement,
                    "tagKey": selected_tag_key if selected_tag_key else "",
                    "tagValue": selected_tag_value if selected_tag_value else ""
                },
                "selected_fields": list(df_to_save.columns),
                "integration_result": {
                    **integration_result,
                    "request_id": str(uuid.uuid4()),
                    "timestamp": datetime.now().isoformat()
                },
                "metadata": {
                    "save_id": save_id,
                    "created_at": datetime.now().isoformat(),
                    "data_version": "1.0",
                    "save_type": "integration_result",
                    "total_rows": len(df_to_save),
                    "total_columns": len(df_to_save.columns)
                }
            }
            
            # JSON 저장
            json_path = os.path.join(save_dir, f"{save_id}.json")
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(json_data, f, indent=2, ensure_ascii=False)
            
            print(f"✓ JSON 저장 완료: {json_path}")
            print(f"\n=== 저장 완료 ===")
            print(f"CSV: {csv_path}")
            print(f"JSON: {json_path}")
        else:
            print("저장할 데이터가 없습니다.")
    else:
        print("통합 데이터 조회 실패")
else:
    print("통합이 실패하여 저장할 수 없습니다.")

저장 ID: integration_20251107_120153_900cdf0b

통합 데이터 조회 중...
조회된 데이터 타입: <class 'dict'>
데이터셋 수: 1
  - farm_protectedHorticulture_westernVegetables_paprika_environment_knu_n: 49497 행

병합된 데이터: 49497 행

✓ CSV 저장 완료: integration_results/integration_20251107_120153_900cdf0b.csv
  저장된 행 수: 49,497
  저장된 컬럼 수: 8
✓ JSON 저장 완료: integration_results/integration_20251107_120153_900cdf0b.json

=== 저장 완료 ===
CSV: integration_results/integration_20251107_120153_900cdf0b.csv
JSON: integration_results/integration_20251107_120153_900cdf0b.json


## 7. 테스트 완료 및 요약

In [22]:
print("="*60)
print("데이터 통합 플랫폼 테스트 완료")
print("="*60)

print("\n✓ 완료된 단계:")
print("1. 기준 데이터 조회 및 선택")
print(f"   - Bucket: {base_bucket}")
print(f"   - Measurement: {base_measurement}")
print(f"   - Tags: {base_tag_key1}={base_tag_value1}, {base_tag_key2}={base_tag_value2}")
print(f"   - 데이터 행 수: 49,497")

print("\n2. 유사 데이터 리스트 조회")
print(f"   - 조회된 유사 데이터: 6개")

print("\n3. 통합할 데이터 선택")
print(f"   - Bucket: {selected_bucket}")
print(f"   - Measurement: {selected_measurement}")
print(f"   - Tags: {selected_tag_key}={selected_tag_value}")

print("\n4. 필드 조회")
print(f"   - 기준 데이터 필드: {len(base_fields)}개")
print(f"   - 통합 데이터 필드: {len(selected_fields)}개")

print("\n5. 데이터 통합 실행")
print(f"   - 상태: {integration_result.get('status')}")
print(f"   - 데이터셋 수: {integration_result.get('result', {}).get('original_datasets')}")
print(f"   - 총 행 수: {integration_result.get('result', {}).get('total_original_rows'):,}")

print("\n6. 결과 저장")
print(f"   - CSV: integration_results/{save_id}.csv")
print(f"   - JSON: integration_results/{save_id}.json")
print(f"   - 저장된 행 수: 49,497")
print(f"   - 저장된 컬럼 수: 8")

print("\n⚠ 주의사항:")
print("- tomato 데이터가 통합 결과에 포함되지 않음")
print("- 원인: 해당 기간(2021-07-01 ~ 2022-10-25)에 tomato 데이터가 없거나")
print("        태그 조건(FARM_ID_CODE=A00259)에 맞는 데이터가 없음")
print("- 기준 데이터(paprika)만 저장됨")

print("\n테스트 기록: test_recording.md 파일 참조")

데이터 통합 플랫폼 테스트 완료

✓ 완료된 단계:
1. 기준 데이터 조회 및 선택
   - Bucket: farm_protectedHorticulture_westernVegetables_paprika
   - Measurement: environment_knu_n
   - Tags: FRM_ID=F00012, ZN_ID=1구역
   - 데이터 행 수: 49,497

2. 유사 데이터 리스트 조회
   - 조회된 유사 데이터: 6개

3. 통합할 데이터 선택
   - Bucket: farm_protectedHorticulture_fruitsAndVegetables_tomato
   - Measurement: environment_knu_sgh
   - Tags: FARM_ID_CODE=A00259

4. 필드 조회
   - 기준 데이터 필드: 8개
   - 통합 데이터 필드: 6개

5. 데이터 통합 실행
   - 상태: success
   - 데이터셋 수: 1
   - 총 행 수: 49,497

6. 결과 저장
   - CSV: integration_results/integration_20251107_120153_900cdf0b.csv
   - JSON: integration_results/integration_20251107_120153_900cdf0b.json
   - 저장된 행 수: 49,497
   - 저장된 컬럼 수: 8

⚠ 주의사항:
- tomato 데이터가 통합 결과에 포함되지 않음
- 원인: 해당 기간(2021-07-01 ~ 2022-10-25)에 tomato 데이터가 없거나
        태그 조건(FARM_ID_CODE=A00259)에 맞는 데이터가 없음
- 기준 데이터(paprika)만 저장됨

테스트 기록: test_recording.md 파일 참조


## 8. Tomato 데이터 미포함 원인 분석

In [23]:
print("="*70)
print("Tomato 데이터 미포함 원인 분석")
print("="*70)

# 1. Tomato 데이터의 시간 범위 확인
print("\n### 1. 데이터 시간 범위 확인 ###")

# Paprika 데이터의 시간 범위
paprika_query = {
    'bucket_name': base_bucket,
    'ms_name': base_measurement,
    'start_time': '2020-01-01 00:00:00',
    'end_time': '2023-12-31 23:59:59',
    'tags': {base_tag_key1: base_tag_value1, base_tag_key2: base_tag_value2}
}

paprika_data = data_interface.get_data_result("ms_by_time", db_client, paprika_query)
if paprika_data is not None and len(paprika_data) > 0:
    print(f"\n[Paprika 데이터]")
    print(f"  시작: {paprika_data.index.min()}")
    print(f"  종료: {paprika_data.index.max()}")
    print(f"  총 행 수: {len(paprika_data):,}")
    print(f"  시간 간격 샘플 (처음 5개):")
    time_diffs = paprika_data.index.to_series().diff().dropna()
    print(f"    {time_diffs.head().tolist()}")
else:
    print("Paprika 데이터 조회 실패")

# Tomato 데이터의 시간 범위 (태그 조건 포함)
print(f"\n[Tomato 데이터 - 태그 조건 포함]")
tomato_query_with_tag = {
    'bucket_name': selected_bucket,
    'ms_name': selected_measurement,
    'start_time': '2020-01-01 00:00:00',
    'end_time': '2023-12-31 23:59:59',
    'tags': {selected_tag_key: selected_tag_value}
}

tomato_data_with_tag = data_interface.get_data_result("ms_by_time", db_client, tomato_query_with_tag)
if tomato_data_with_tag is not None and len(tomato_data_with_tag) > 0:
    print(f"  시작: {tomato_data_with_tag.index.min()}")
    print(f"  종료: {tomato_data_with_tag.index.max()}")
    print(f"  총 행 수: {len(tomato_data_with_tag):,}")
    print(f"  시간 간격 샘플 (처음 5개):")
    time_diffs = tomato_data_with_tag.index.to_series().diff().dropna()
    print(f"    {time_diffs.head().tolist()}")
else:
    print(f"  ❌ 태그 조건 ({selected_tag_key}={selected_tag_value})으로 데이터 없음!")

# Tomato 데이터의 시간 범위 (태그 조건 없이)
print(f"\n[Tomato 데이터 - 태그 조건 없이]")
tomato_query_no_tag = {
    'bucket_name': selected_bucket,
    'ms_name': selected_measurement,
    'start_time': '2020-01-01 00:00:00',
    'end_time': '2023-12-31 23:59:59',
    'tags': {}
}

tomato_data_no_tag = data_interface.get_data_result("ms_by_time", db_client, tomato_query_no_tag)
if tomato_data_no_tag is not None and len(tomato_data_no_tag) > 0:
    print(f"  시작: {tomato_data_no_tag.index.min()}")
    print(f"  종료: {tomato_data_no_tag.index.max()}")
    print(f"  총 행 수: {len(tomato_data_no_tag):,}")
    print(f"  시간 간격 샘플 (처음 5개):")
    time_diffs = tomato_data_no_tag.index.to_series().diff().dropna()
    print(f"    {time_diffs.head().tolist()}")
    print(f"  컬럼: {list(tomato_data_no_tag.columns)}")
else:
    print(f"  ❌ 태그 없이도 데이터 없음!")

Tomato 데이터 미포함 원인 분석

### 1. 데이터 시간 범위 확인 ###

[Paprika 데이터]
  시작: 2021-07-01 00:00:00+00:00
  종료: 2022-10-25 11:10:00+00:00
  총 행 수: 49,497
  시간 간격 샘플 (처음 5개):
    [Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00')]

[Tomato 데이터 - 태그 조건 포함]
  시작: 2023-06-01 00:00:00+00:00
  종료: 2023-09-01 00:00:00+00:00
  총 행 수: 13,249
  시간 간격 샘플 (처음 5개):
    [Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00')]

[Tomato 데이터 - 태그 조건 없이]
  시작: 2023-06-01 00:00:00+00:00
  종료: 2023-09-01 00:00:00+00:00
  총 행 수: 13,249
  시간 간격 샘플 (처음 5개):
    [Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00'), Timedelta('0 days 00:10:00')]
  컬럼: ['FARM_ID_CODE', 'ACC_RADIATION_INTENSITY', 'EXT_SLR_RADIATON', 'INT_CO2', 'INT_REL_HD', 'MID_INT_TMP', 'SLR_RAD']


In [24]:
print("\n" + "="*70)
print("### 2. 원인 분석 결과 ###")
print("="*70)

print("\n✅ 발견된 사실:")
print(f"1. Paprika 데이터 기간: 2021-07-01 ~ 2022-10-25 (약 16개월)")
print(f"2. Tomato 데이터 기간:  2023-06-01 ~ 2023-09-01 (약 3개월)")
print(f"3. 두 데이터의 시간 간격: 동일 (10분)")

print("\n❌ 문제 원인:")
print("━" * 70)
print("두 데이터의 시간 범위가 전혀 겹치지 않음!")
print("━" * 70)
print(f"  • Paprika 종료: 2022-10-25")
print(f"  • Tomato 시작:  2023-06-01")
print(f"  • 간격: 약 7개월 차이")

print("\n💡 결론:")
print("  → 주기(10분)는 동일하지만, 시간 범위가 달라서 통합 불가")
print("  → 요청한 통합 기간(2021-07-01 ~ 2022-10-25)에 Tomato 데이터가")
print("     존재하지 않기 때문에 통합 결과에 포함되지 않음")

print("\n🔧 해결 방안:")
print("  1. Tomato 데이터가 있는 기간으로 변경: 2023-06-01 ~ 2023-09-01")
print("  2. 다른 Tomato 데이터셋 찾기 (2021-2022년 데이터)")
print("  3. Paprika와 시간이 겹치는 다른 데이터 선택")

# 시각적으로 표현
print("\n📊 시간 범위 비교:")
print("  2021-07  2021-12  2022-06  2022-12  2023-06  2023-09")
print("  |━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━|")
print("  [   Paprika 데이터 (16개월)   ]")
print("                                         [Tomato 데이터]")
print("                                  (7개월 공백)")


### 2. 원인 분석 결과 ###

✅ 발견된 사실:
1. Paprika 데이터 기간: 2021-07-01 ~ 2022-10-25 (약 16개월)
2. Tomato 데이터 기간:  2023-06-01 ~ 2023-09-01 (약 3개월)
3. 두 데이터의 시간 간격: 동일 (10분)

❌ 문제 원인:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
두 데이터의 시간 범위가 전혀 겹치지 않음!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  • Paprika 종료: 2022-10-25
  • Tomato 시작:  2023-06-01
  • 간격: 약 7개월 차이

💡 결론:
  → 주기(10분)는 동일하지만, 시간 범위가 달라서 통합 불가
  → 요청한 통합 기간(2021-07-01 ~ 2022-10-25)에 Tomato 데이터가
     존재하지 않기 때문에 통합 결과에 포함되지 않음

🔧 해결 방안:
  1. Tomato 데이터가 있는 기간으로 변경: 2023-06-01 ~ 2023-09-01
  2. 다른 Tomato 데이터셋 찾기 (2021-2022년 데이터)
  3. Paprika와 시간이 겹치는 다른 데이터 선택

📊 시간 범위 비교:
  2021-07  2021-12  2022-06  2022-12  2023-06  2023-09
  |━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━|
  [   Paprika 데이터 (16개월)   ]
                                         [Tomato 데이터]
                                  (7개월 공백)


## 9. 대안: Tomato 데이터 기간으로 재통합 테스트

In [25]:
# Tomato 데이터가 있는 기간으로 재통합 (선택사항)
print("="*70)
print("대안: Tomato 데이터 기간으로 재통합")
print("="*70)
print("\n참고: 이 셀은 실험적 테스트입니다.")
print("Paprika 데이터는 2023년에 없으므로 Tomato만 포함될 것으로 예상됩니다.")
print("\n실행하려면 아래 코드의 주석을 제거하세요.")

# 주석 제거하고 실행하면 Tomato 데이터 기간으로 통합 시도
"""
alternative_request = {
    'base_data': [
        base_measurement,
        base_bucket,
        base_tag_key1,
        base_tag_value1,
        []
    ],
    'selected_data': [
        selected_measurement,
        selected_bucket,
        selected_tag_key,
        selected_tag_value,
        []
    ],
    'start_time': '2023-06-01',  # Tomato 데이터 시작
    'end_time': '2023-09-01',    # Tomato 데이터 종료
    'Period': 'data2'  # Tomato 기준
}

print("\n통합 실행 중 (2023-06-01 ~ 2023-09-01)...")
alt_result = process_data_integration(alternative_request)

print(f"\n상태: {alt_result.get('status')}")
print(f"메시지: {alt_result.get('message')}")

if alt_result.get('status') == 'success':
    result = alt_result.get('result', {})
    print(f"\n데이터셋 수: {result.get('original_datasets')}")
    print(f"총 행 수: {result.get('total_original_rows'):,}")
    for ds_name, count in result.get('dataset_counts', {}).items():
        print(f"  - {ds_name}: {count:,}행")
"""

print("\n✅ 원인 분석 완료!")
print("자세한 내용은 test_recording.md를 참조하세요.")

대안: Tomato 데이터 기간으로 재통합

참고: 이 셀은 실험적 테스트입니다.
Paprika 데이터는 2023년에 없으므로 Tomato만 포함될 것으로 예상됩니다.

실행하려면 아래 코드의 주석을 제거하세요.

✅ 원인 분석 완료!
자세한 내용은 test_recording.md를 참조하세요.


## 10. Paprika와 시간 범위가 겹치는 Farm 데이터 찾기

In [27]:
print("="*70)
print("Paprika와 시간 범위가 겹치고 유사도가 높은 데이터 찾기")
print("="*70)

# Paprika 데이터 기간
paprika_start = pd.Timestamp('2021-07-01', tz='UTC')
paprika_end = pd.Timestamp('2022-10-25', tz='UTC')

print(f"\n[기준] Paprika 데이터 기간:")
print(f"  시작: {paprika_start}")
print(f"  종료: {paprika_end}")
print(f"  기간: {(paprika_end - paprika_start).days}일")

# 유사 데이터 리스트에서 시간 범위 확인
print("\n" + "="*70)
print("유사 데이터 리스트에서 시간 범위 확인")
print("="*70)

candidates = []

for idx, data in enumerate(all_secondary_list):
    bucket = data[0]
    measurement = data[1]
    tags = data[2] if len(data) > 2 else {}
    score = data[3] if len(data) > 3 else 0
    spatial_sim = data[4] if len(data) > 4 else False
    keyword_sim = data[5] if len(data) > 5 else False
    
    print(f"\n[{idx+1}] {bucket} / {measurement}")
    print(f"  Score: {score}, Spatial: {spatial_sim}, Keyword: {keyword_sim}")
    print(f"  Tags: {tags}")
    
    try:
        # 태그 정보 추출
        tag_key = list(tags.keys())[0] if tags else None
        tag_value = tags[tag_key] if tag_key else None
        
        # 데이터 조회
        query = {
            'bucket_name': bucket,
            'ms_name': measurement,
            'start_time': '2020-01-01 00:00:00',
            'end_time': '2023-12-31 23:59:59',
            'tags': {tag_key: tag_value} if tag_key else {}
        }
        
        candidate_data = data_interface.get_data_result("ms_by_time", db_client, query)
        
        if candidate_data is not None and len(candidate_data) > 0:
            data_start = candidate_data.index.min()
            data_end = candidate_data.index.max()
            
            print(f"  시작: {data_start}")
            print(f"  종료: {data_end}")
            
            # 시간 범위 겹침 확인
            overlap_start = max(paprika_start, data_start)
            overlap_end = min(paprika_end, data_end)
            
            if overlap_start < overlap_end:
                overlap_days = (overlap_end - overlap_start).days
                overlap_ratio = overlap_days / (paprika_end - paprika_start).days * 100
                
                print(f"  ✅ 겹치는 기간: {overlap_days}일 ({overlap_ratio:.1f}%)")
                print(f"  행 수: {len(candidate_data):,}")
                
                # 시간 간격 확인
                time_diffs = candidate_data.index.to_series().diff().dropna()
                avg_interval = time_diffs.mean()
                print(f"  평균 시간 간격: {avg_interval}")
                
                candidates.append({
                    'rank': idx + 1,
                    'bucket': bucket,
                    'measurement': measurement,
                    'tags': tags,
                    'score': score,
                    'spatial_sim': spatial_sim,
                    'keyword_sim': keyword_sim,
                    'start': data_start,
                    'end': data_end,
                    'overlap_days': overlap_days,
                    'overlap_ratio': overlap_ratio,
                    'row_count': len(candidate_data),
                    'avg_interval': avg_interval
                })
            else:
                print(f"  ❌ 겹침 없음")
        else:
            print(f"  ❌ 데이터 없음")
            
    except Exception as e:
        print(f"  ⚠️  조회 오류: {str(e)[:80]}")

# 결과 정리
print("\n" + "="*70)
print("✓ 검색 완료")
print("="*70)

if candidates:
    print(f"\n총 {len(candidates)}개의 겹치는 데이터 발견!\n")
    
    # 정렬: 겹치는 비율 > 공간 유사도 > 스코어 순
    candidates.sort(key=lambda x: (x['overlap_ratio'], x['spatial_sim'], x['score']), reverse=True)
    
    print("추천 순위 (겹침 비율 > 위치 유사도 > 스코어 순):")
    print("-" * 70)
    
    for i, c in enumerate(candidates):
        print(f"\n{i+1}. {c['bucket']} / {c['measurement']}")
        print(f"   겹침: {c['overlap_ratio']:.1f}% ({c['overlap_days']}일)")
        print(f"   Score: {c['score']}")
        print(f"   위치 유사도: {'✓' if c['spatial_sim'] else '×'}")
        print(f"   키워드 유사도: {'✓' if c['keyword_sim'] else '×'}")
        print(f"   Tags: {c['tags']}")
        print(f"   행 수: {c['row_count']:,}")
        print(f"   평균 간격: {c['avg_interval']}")
else:
    print("\n❌ 시간 범위가 겹치는 데이터가 없습니다.")

Paprika와 시간 범위가 겹치고 유사도가 높은 데이터 찾기

[기준] Paprika 데이터 기간:
  시작: 2021-07-01 00:00:00+00:00
  종료: 2022-10-25 00:00:00+00:00
  기간: 481일

유사 데이터 리스트에서 시간 범위 확인

[1] farm_protectedHorticulture_vegetables_cucumber / environment_knu_sgh
  Score: 0.75, Spatial: False, Keyword: False
  Tags: {'FARM_ID_CODE': 'A00259'}
  시작: 2021-10-19 00:00:00+00:00
  종료: 2022-05-02 23:59:00+00:00
  ✅ 겹치는 기간: 195일 (40.5%)
  행 수: 282,240
  평균 시간 간격: 0 days 00:01:00

[2] farm_protectedHorticulture_vegetables_cucumber / yield_knu_n
  Score: 0.80, Spatial: False, Keyword: True
  Tags: {'FARM_ID_CODE': 'A00259'}
  시작: 2023-11-25 00:00:00+00:00
  종료: 2023-12-31 00:00:00+00:00
  ❌ 겹침 없음

[3] farm_protectedHorticulture_fruitsAndVegetables_tomato / environment_knu_n
  Score: 0.82, Spatial: False, Keyword: False
  Tags: {'FRM_ID': 'F00013', 'ZN_ID': '1구역'}
  시작: 2021-07-01 00:00:00+00:00
  종료: 2021-11-26 23:00:00+00:00
  ✅ 겹치는 기간: 148일 (30.8%)
  행 수: 3,576
  평균 시간 간격: 0 days 01:00:00

[4] farm_protectedHorticulture_fruits

In [28]:
# 최적의 통합 대상 선택
print("\n" + "="*70)
print("최적의 통합 대상 선택")
print("="*70)

if candidates:
    # 가장 좋은 후보 선택
    best_candidate = candidates[0]
    
    print(f"\n✨ 추천 데이터:")
    print(f"   Bucket: {best_candidate['bucket']}")
    print(f"   Measurement: {best_candidate['measurement']}")
    print(f"   Tags: {best_candidate['tags']}")
    print(f"   겹침: {best_candidate['overlap_ratio']:.1f}% ({best_candidate['overlap_days']}일)")
    print(f"   Score: {best_candidate['score']}")
    print(f"   위치 유사도: {'✓' if best_candidate['spatial_sim'] else '×'}")
    print(f"   키워드 유사도: {'✓' if best_candidate['keyword_sim'] else '×'}")
    print(f"   행 수: {best_candidate['row_count']:,}")
    
    # 변수 업데이트
    selected_bucket = best_candidate['bucket']
    selected_measurement = best_candidate['measurement']
    selected_tags = best_candidate['tags']
    selected_tag_key = list(selected_tags.keys())[0] if selected_tags else None
    selected_tag_value = selected_tags[selected_tag_key] if selected_tag_key else None
    
    print(f"\n변수 업데이트 완료!")
    print(f"  selected_bucket: {selected_bucket}")
    print(f"  selected_measurement: {selected_measurement}")
    print(f"  selected_tag_key: {selected_tag_key}")
    print(f"  selected_tag_value: {selected_tag_value}")
    
    # Top 3 출력
    if len(candidates) > 1:
        print(f"\n📋 다른 후보들:")
        for i, c in enumerate(candidates[1:4], 2):
            print(f"\n{i}. {c['bucket']} / {c['measurement']}")
            print(f"   겹침: {c['overlap_ratio']:.1f}%, Score: {c['score']}, "
                  f"위치: {'✓' if c['spatial_sim'] else '×'}")
else:
    print("\n❌ 적합한 후보가 없습니다.")


최적의 통합 대상 선택

✨ 추천 데이터:
   Bucket: farm_protectedHorticulture_westernVegetables_paprika
   Measurement: machineControl_knu_n
   Tags: {'FRM_ID': 'F00012', 'ZN_ID': '1구역'}
   겹침: 99.8% (480일)
   Score: 0.78
   위치 유사도: ✓
   키워드 유사도: ✓
   행 수: 489,567

변수 업데이트 완료!
  selected_bucket: farm_protectedHorticulture_westernVegetables_paprika
  selected_measurement: machineControl_knu_n
  selected_tag_key: FRM_ID
  selected_tag_value: F00012

📋 다른 후보들:

2. farm_protectedHorticulture_vegetables_cucumber / environment_knu_sgh
   겹침: 40.5%, Score: 0.75, 위치: ×

3. farm_protectedHorticulture_fruitsAndVegetables_tomato / environment_knu_n
   겹침: 30.8%, Score: 0.82, 위치: ×


## 11. 선택된 데이터로 재통합 실행

In [32]:
# 선택된 데이터의 필드 조회
selected_query_json = {
    'bucket_name': selected_bucket,
    'ms_name': selected_measurement,
    'start_time': '2021-07-01 00:00:00',
    'end_time': '2022-10-25 23:59:59',
    'tags': {selected_tag_key: selected_tag_value}
}

# 선택된 데이터 조회
selected_data = data_interface.get_data_result("ms_by_time", db_client, selected_query_json)
print(f"\n선택된 데이터 조회 완료")
print(f"데이터 개수: {len(selected_data)} rows")
print(f"기간: {selected_data.index[0]} ~ {selected_data.index[-1]}")

# 선택된 데이터의 필드 목록
selected_fields = [col for col in selected_data.columns if col not in ['bucket', 'measurement']]
print(f"\n선택된 데이터 필드 ({len(selected_fields)}개): {selected_fields}")

# 기준 데이터 필드 목록 (이미 커널에 저장되어 있음)
print(f"\n기준 데이터 필드 ({len(base_fields)}개): {base_fields}")


선택된 데이터 조회 완료
데이터 개수: 489567 rows
기간: 2021-07-01 00:00:02+00:00 ~ 2022-10-25 12:01:00+00:00

선택된 데이터 필드 (12개): ['ZN_ID', 'FIRST_LW_OP_RATE', 'FIRST_RW_OP_RATE', 'HV_OP_RATE', 'LAG_CRTN_OP_RATE', 'SECOND_LW_OP_RATE', 'SECOND_RW_OP_RATE', 'SECOND_TW_LEFT_OP_RATE', 'SECOND_TW_RIGHT_OP_RATE', 'SH_CRTN_OP_RATE', 'S_CRTN_OP_RATE', 'TOP_CRTN_OP_RATE']

기준 데이터 필드 (8개): ['EXT_TMP', 'EXT_WIND_DIRECT', 'EXT_WIND_SPEED', 'INT_CO2', 'INT_REL_HD', 'INT_TMP', 'MAX_EXT_TMP', 'MIN_EXT_TMP']


In [39]:
# 재통합 실행
print("\n=== 데이터 통합 실행 ===")
print(f"기준 데이터: {base_bucket}/{base_measurement}")
print(f"통합 데이터: {selected_bucket}/{selected_measurement}")

# 통합 요청 데이터 구성 (API 형식: [measurement, bucket, tag_key, tag_value, features])
integration_request = {
    'base_data': [
        base_measurement,  # measurement
        base_bucket,       # bucket
        base_tag_key1,     # tag_key
        base_tag_value1,   # tag_value
        []                 # features - 빈 리스트로 모든 필드 사용
    ],
    'selected_data': [
        selected_measurement,  # measurement
        selected_bucket,       # bucket
        selected_tag_key,      # tag_key
        selected_tag_value,    # tag_value
        []                     # features - 빈 리스트로 모든 필드 사용
    ],
    'start_time': '2021-07-01',
    'end_time': '2022-10-25',
    'Period': 'data1'  # 기준 데이터의 주기 사용
}

print(f"\n통합 파라미터:")
print(f"- 기준 데이터: {base_bucket}/{base_measurement}")
print(f"  태그: {base_tag_key1}={base_tag_value1}")
print(f"- 통합 데이터: {selected_bucket}/{selected_measurement}")
print(f"  태그: {selected_tag_key}={selected_tag_value}")
print(f"- 기간: {integration_request['start_time']} ~ {integration_request['end_time']}")
print(f"- 주기: {integration_request['Period']}")

# 통합 실행
print("\n통합 실행 중...")
integration_result = process_data_integration(integration_request)

print(f"\n통합 완료!")
print(f"Status: {integration_result.get('status', 'N/A')}")


=== 데이터 통합 실행 ===
기준 데이터: farm_protectedHorticulture_westernVegetables_paprika/environment_knu_n
통합 데이터: farm_protectedHorticulture_westernVegetables_paprika/machineControl_knu_n

통합 파라미터:
- 기준 데이터: farm_protectedHorticulture_westernVegetables_paprika/environment_knu_n
  태그: FRM_ID=F00012
- 통합 데이터: farm_protectedHorticulture_westernVegetables_paprika/machineControl_knu_n
  태그: FRM_ID=F00012
- 기간: 2021-07-01 ~ 2022-10-25
- 주기: data1

통합 실행 중...

통합 완료!
Status: success


In [ ]:
# 통합 결과 상세 확인
print("\n=== 통합 결과 상세 ===")
result = integration_result['result']

print(f"\n✅ 데이터 통합 성공!")
print(f"Status: {integration_result['status']}")
print(f"Message: {integration_result['message']}")

print(f"\n📊 원본 데이터셋:")
for ds_name, count in result['dataset_counts'].items():
    print(f"   - {ds_name}: {count:,} rows")
print(f"   📍 총 원본 행 수: {result['total_original_rows']:,}")

print(f"\n🕐 통합 정보:")
print(f"   - 시간 범위: {result['time_range']['start']} ~ {result['time_range']['end']}")
print(f"   - 통합 주기: {result['integration_frequency']}")

print(f"\n? 통합 통계:")
for key, value in result['integrated_stats'].items():
    print(f"   - {key}: {value}")

print(f"\n🔍 result 전체 키:")
print(f"   {list(result.keys())}")

print(f"\n🔍 original_datasets:")
print(f"   타입: {type(result.get('original_datasets', 'N/A'))}")
print(f"   값: {result.get('original_datasets', 'N/A')}")

In [ ]:
# 재통합 결과 저장
import datetime
import uuid

# 타임스탬프 및 고유 ID 생성
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
unique_id = uuid.uuid4().hex[:8]
filename_base = f"integration_{timestamp}_{unique_id}"

# 저장 디렉토리
save_dir = "../integration_results"
os.makedirs(save_dir, exist_ok=True)

# CSV 저장
csv_path = os.path.join(save_dir, f"{filename_base}.csv")
integration_result[0]['data'].to_csv(csv_path, encoding='utf-8-sig')
print(f"CSV 저장 완료: {csv_path}")

# JSON 메타데이터 저장
json_data = {
    "integration_info": {
        "timestamp": timestamp,
        "unique_id": unique_id,
        "base_data": {
            "bucket": base_bucket,
            "measurement": base_measurement,
            "tags": {base_tag_key: base_tag_value}
        },
        "integrated_data": [{
            "bucket": selected_bucket,
            "measurement": selected_measurement,
            "tags": {selected_tag_key: selected_tag_value}
        }],
        "time_range": {
            "start": integration_params['start_time'],
            "end": integration_params['end_time']
        },
        "period": integration_params['integration_period']
    },
    "results": []
}

for result in integration_result:
    json_data["results"].append({
        "bucket": result['bucket'],
        "measurement": result['measurement'],
        "tags": result['tags'],
        "row_count": len(result['data']),
        "columns": list(result['data'].columns),
        "time_range": {
            "start": result['data'].index[0].isoformat(),
            "end": result['data'].index[-1].isoformat()
        }
    })

json_path = os.path.join(save_dir, f"{filename_base}.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)
print(f"JSON 저장 완료: {json_path}")

print(f"\n저장된 파일:")
print(f"- CSV: {filename_base}.csv")
print(f"- JSON: {filename_base}.json")